# Neural Language Models

- 📺 **Video:** [https://youtu.be/59NrmwAdOWA](https://youtu.be/59NrmwAdOWA)

## Overview
Transitions to Neural Network-based language models, which overcome some limitations of n-gram models. The video likely recalls that in Week 3 we learned how to represent words with embeddings; now we can feed those into a neural sequence model to predict the next word.

In [ ]:
import os, random
random.seed(0)
CI = os.environ.get('CI') == 'true'

## Key ideas
- It introduces the idea of using an RNN (Recurrent Neural Network) or similar architecture for LM.
- Perhaps it starts with the older model by Bengio et al.
- (2003) - a feedforward neural LM that used a fixed context (like 4 previous words) but projected them into embeddings and used a hidden layer to predict the next word, dramatically reducing parameters via shared embeddings.
- Then it leads into Recurrent Neural Networks, which don't have a fixed context window but instead maintain a hidden state that carries information from all prior words (in theory).

## Demo

In [ ]:
print('Try the exercises below and follow the linked materials.')

## Try it
- Modify the demo
- Add a tiny dataset or counter-example


## References
- [Eisenstein 6.1](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Eisenstein 6.2](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Eisenstein 6.4](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Eisenstein 6.3](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [[Blog] Understanding LSTMs](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)
- [[Blog] The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)
- [Train Short, Test Long: Attention with Linear Biases Enables Input Length Extrapolation](https://arxiv.org/abs/2108.12409)
- [The Impact of Positional Encoding on Length Generalization in Transformers](https://arxiv.org/abs/2305.19466)


*Links only; we do not redistribute slides or papers.*